In [2]:
# Initialization and Parameter setting

import numpy as np
import matplotlib.pyplot as pp
import pycbc.noise
import pycbc.psd
from scipy import signal
import time
from PIL import Image
import pickle
import os

# Parameters for noise generation
flow = 30.0
delta_f = 1.0 / 16
flen = int(8192 / delta_f) + 1
psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

# Generate 8 seconds of noise at 16384 Hz
delta_t = 1.0 / 16384
tsamples = int(8 / delta_t) # 8 sec

# Sampling frequency, window, and FFT parameters for the spectrogram
fs = 16384  # Sampling frequency (Hz) - will adjust the frequency, but not the total value
window_size = 16384  # Window size (Blackman window)
step_size = 1024  # Step size (determines overlap)
nfft = 16384  # Number of points in FFT (should match window size)

# Directory to save the pickle file
folder_name = "noise_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    



In [31]:
total_time_spec_calc = 0
total_time_image_gen = 0

# The number of images to generate/save
number_of_images = 1000

# List to store spectrogram image data
noise_images_1000 = [] # The number should be equal to the number of images

start_time_2 = time.time()

# Loop to generate N number of different noise signals
for i in range(number_of_images):
    seed = 100 + i # Vary the seed for each noise
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd, seed=seed)
    
    # Start timer for spectrogram calculation
    start_time_0 = time.time()
    
    # Generate the spectrogram
    frequencies, times, Sxx = signal.spectrogram(
        ts,  # Convert PyCBC time series to numpy array  
        fs=fs, # Sampling frequency
        window='blackman',  # Use Blackman window
        nperseg=window_size,  # Size of each window
        noverlap=window_size - step_size,  # overlap between segments
        nfft=nfft,  # Number of points for FFT
    )
    
    # End timer for spectrogram calculation
    end_time_0 = time.time()
    time_0 = end_time_0 - start_time_0
    # Calculate the total time for spectrogram calculation
    total_time_spec_calc = total_time_spec_calc + time_0
    
    # Start timer for image generation
    start_time_1 = time.time()
    
    # Initialization for the low and high frequencies
    fhi = 150.0
    freq_dim = np.arange(len(frequencies))
    freq = freq_dim[(frequencies >= flow) & (frequencies <= fhi)]
    Sxx_fhl = Sxx[freq]
    
    # Normalize the spectrogram to the range [0, 1] and scale to [0, 255]
    Sxx_fhl_log = np.log10(Sxx_fhl)
    Sxx_fhl_log_min, Sxx_fhl_log_max = Sxx_fhl_log.min(), Sxx_fhl_log.max()
    Sxx_fhl_log_norm = (Sxx_fhl_log - Sxx_fhl_log_min)/(Sxx_fhl_log_max - Sxx_fhl_log_min)
    Sxx_fhl_log_scaled = (Sxx_fhl_log_norm*255).astype(np.uint8) # Scaled from 0 to 255
    Sxx_log_rgb = np.stack([Sxx_fhl_log_scaled, np.zeros_like(Sxx_fhl_log_scaled), np.zeros_like(Sxx_fhl_log_scaled)], axis = -1)
    
    noise_images_1000.append(Sxx_fhl_log_scaled) # The number in the list name should be the same with the number of images
    
    end_time_1 = time.time()
    time_1 = end_time_1 - start_time_1
    total_time_image_gen = total_time_image_gen + time_1
    
# Save the list of RGB Spectrogram images as pickle file
pickle_filename = os.path.join(folder_name, f"{i+1}_Grayscale_noise_ps_20241003.pkl")
with open(pickle_filename, 'wb') as f:
    pickle.dump(noise_images_1000, f) # The number in the object name should be the same with the number of images
    
end_time_2 = time.time()
total_time_save_pickle = end_time_2 - start_time_2
    
print(f"All RGB Spectrogram images saved in '{pickle_filename}'")
print(f"Total time for calculating the spectrogram: {total_time_spec_calc:.2f} seconds")
print(f"Average time in calculating the spectrogram: {total_time_spec_calc/i:.2f} second")
print(f"Total time to generate the images: {total_time_image_gen:.2f} seconds")
print(f"Total time to pickle: {total_time_save_pickle:.2f} seconds")


All RGB Spectrogram images saved in 'noise_images/1000_Grayscale_noise_ps_20241003.pkl'
Total time for calculating the spectrogram: 32.50 seconds
Average time in calculating the spectrogram: 0.03 second
Total time to generate the images: 0.34 seconds
Total time to pickle: 118.07 seconds


In [23]:
noise_images_10 = []